In [3]:
import tbapy
import pprint
pp = pprint.PrettyPrinter()

api_key = "6qOZ9uAEsb4CDrOBNG6ZnIdi9cWBaZ6DHnCSato97Qfo7bBeUwT9NfFt4Gi5sHFN"

tba = tbapy.TBA(api_key)
pp.pprint(tba.status())

{'android': {'latest_app_version': 4020399, 'min_app_version': -1},
 'backup': {},
 'contbuild_enabled': True,
 'current_season': 2017,
 'down_events': [],
 'ios': {'latest_app_version': -1, 'min_app_version': -1},
 'is_datafeed_down': False,
 'json': {'android': {'latest_app_version': 4020399, 'min_app_version': -1},
          'backup': {},
          'contbuild_enabled': True,
          'current_season': 2017,
          'down_events': [],
          'ios': {'latest_app_version': -1, 'min_app_version': -1},
          'is_datafeed_down': False,
          'max_season': 2018,
          'web': {'commit_time': '2017-12-23 23:38:18 -0800',
                  'current_commit': 'f76caf44eb4f476e3dbc7ba83b6ed8a24c781a65',
                  'deploy_time': 'Sun Dec 24 07:54:40 UTC 2017',
                  'travis_job': '320951454'}},
 'max_season': 2018,
 'web': {'commit_time': '2017-12-23 23:38:18 -0800',
         'current_commit': 'f76caf44eb4f476e3dbc7ba83b6ed8a24c781a65',
         'deploy_time'

In [199]:
"Memoize my favorite functions"

from functools import lru_cache

team_events = lru_cache(maxsize=512)(tba.team_events)
event_alliances = lru_cache(maxsize=512)(tba.event_alliances)
event_rankings = lru_cache(maxsize=512)(tba.event_rankings)
event_oprs = lru_cache(maxsize=512)(tba.event_oprs)

def scrape_event_codes(events, year):
    return [str(year) + event['event_code'] for event in events]

pp.pprint(scrape_event_codes(team_events('frc973', 2017), 2017))

['2017cabl',
 '2017cacc',
 '2017cada',
 '2017cama',
 '2017cc',
 '2017cmptx',
 '2017mttd',
 '2017nhfoc',
 '2017onsc',
 '2017roe']


In [140]:
pp.pprint(event_rankings('2017cc'))

{'extra_stats_info': [{'name': 'Total Ranking Points', 'precision': 0}],
 'json': {'extra_stats_info': [{'name': 'Total Ranking Points',
                                'precision': 0}],
          'rankings': [{'dq': 0,
                        'extra_stats': [25],
                        'matches_played': 10,
                        'qual_average': None,
                        'rank': 1,
                        'record': {'losses': 3, 'ties': 0, 'wins': 7},
                        'sort_orders': [2.5,
                                        3335.0,
                                        759.0,
                                        1420.0,
                                        1250.0,
                                        465.0],
                        'team_key': 'frc254'},
                       {'dq': 0,
                        'extra_stats': [22],
                        'matches_played': 10,
                        'qual_average': None,
                        'rank': 2,
 

In [5]:
#pp.pprint(tba.event_alliances("2017cacc"))
from functools import reduce

def scrape_team_results(finals_report, team):
    seat_count = {
        'captain': 0,
        'first': 0,
        'second': 0,
        'third': 0,
    }
    seat_order = ['captain', 'first', 'second', 'third']
    
    level_count = {
        'w': 0,
        'f': 0,
        'sf': 0,
        'qf': 0,
    }
    
    if finals_report is None:
        return None
    
    for alliance in finals_report:
        if team not in alliance['picks'] or alliance['status'] == 'unknown':
            continue
            
        seat = alliance['picks'].index(team)
        seat_count[seat_order[seat]] += 1
        level = 'w' if alliance['status']['status'] == 'won' else alliance['status']['level']
        level_count[level] += 1
    
    return {
        'seat_count': seat_count,
        'level_count': level_count,
    }

def merge_team_results(res_a, res_b):
    def sum_merge(dict_a, dict_b):
        return { k: dict_a.get(k, 0) + dict_b.get(k, 0) for k in set(dict_a) | set(dict_b) }
    
    if res_a is None or res_b is None:
        return None
    
    return {
        'seat_count': sum_merge(res_a['seat_count'], res_b['seat_count']),
        'level_count': sum_merge(res_a['level_count'], res_b['level_count']),
    }

def score_team_results(results, num_events):
    return (num_events +
            (3 * results['level_count']['w']
             + 2 * results['level_count']['f']
             + 1 * results['level_count']['sf']
             + 0.5 * results['level_count']['qf']
             + 5 * results['seat_count']['captain']
             + 3 * results['seat_count']['first']
             + 2 * results['seat_count']['second']
             + 0.5 * results['seat_count']['third']) / float(num_events))

def prescout_team_year(team, year):
    events = scrape_event_codes(team_events(team, year), year)
    
    event_results = []
    for event in events:
        try:
            event_results.append(scrape_team_results(event_alliances(event), team))
        except TypeError:
            pass
        
    if len(event_results) == 0:
        return None
        
    merged_event_results = reduce(merge_team_results, event_results)
    return {
        'seat_count': merged_event_results['seat_count'],
        'level_count': merged_event_results['level_count'],
        'events': events,
        'event_count': len(events),
        'numeric_score': score_team_results(merged_event_results, len(events))
    }

pp.pprint(prescout_team_year('frc973', 2017))
pp.pprint(prescout_team_year('frc5012', 2015))
pp.pprint(prescout_team_year('frc5012', 2013))

{'event_count': 10,
 'events': ['2017cabl',
            '2017cacc',
            '2017cada',
            '2017cama',
            '2017cc',
            '2017cmptx',
            '2017mttd',
            '2017nhfoc',
            '2017onsc',
            '2017roe'],
 'level_count': {'f': 1, 'qf': 0, 'sf': 2, 'w': 6},
 'numeric_score': 15.75,
 'seat_count': {'captain': 6, 'first': 1, 'second': 1, 'third': 1}}
{'event_count': 6,
 'events': ['2015carm', '2015cc', '2015cmp', '2015mttd', '2015new', '2015nvlv'],
 'level_count': {'f': 0, 'qf': 1, 'sf': 1, 'w': 2},
 'numeric_score': 8.25,
 'seat_count': {'captain': 0, 'first': 1, 'second': 1, 'third': 2}}
None


In [6]:
MAX_YEAR = 2017

def prescout_team(team, years_past=4, max_year=MAX_YEAR):
    results = {}
    
    for i in range(years_past):
        results[max_year - i] = prescout_team_year(team, max_year - i)
    
    return results

pp.pprint(prescout_team('frc5012'))

{2014: {'event_count': 5,
        'events': ['2014cafc2', '2014casb', '2014cc', '2014gal', '2014nvlv'],
        'level_count': {'f': 0, 'qf': 0, 'sf': 1, 'w': 0},
        'numeric_score': 5.6,
        'seat_count': {'captain': 0, 'first': 0, 'second': 1, 'third': 0}},
 2015: {'event_count': 6,
        'events': ['2015carm',
                   '2015cc',
                   '2015cmp',
                   '2015mttd',
                   '2015new',
                   '2015nvlv'],
        'level_count': {'f': 0, 'qf': 1, 'sf': 1, 'w': 2},
        'numeric_score': 8.25,
        'seat_count': {'captain': 0, 'first': 1, 'second': 1, 'third': 2}},
 2016: {'event_count': 5,
        'events': ['2016cabb', '2016calb', '2016cc', '2016gal', '2016nvlv'],
        'level_count': {'f': 0, 'qf': 0, 'sf': 3, 'w': 1},
        'numeric_score': 9.0,
        'seat_count': {'captain': 2, 'first': 0, 'second': 2, 'third': 0}},
 2017: {'event_count': 6,
        'events': ['2017cada',
                   '2017casd',


In [21]:
def agr(lst):
    res = lst[0]
    for score in lst[1:]:
        res = (res + float(score)) / 2.0
    return res

class ProgressTracker:
    def __init__(self, total):
        self.call_count = 0
        self.total = total
        
    def done(self):
        self.call_count += 1
        percent_done = 100.0 * self.call_count / self.total
        prev_percent_done = 100.0 * (self.call_count - 1) / self.total
        curr_mark = int(percent_done) // 10
        prev_mark = int(prev_percent_done) // 10
        if curr_mark > prev_mark:
            print("{:2.0f} percent done ({:d} out of {:d})".format(percent_done, self.call_count, self.total))
        

def prescout_teams(teams):
    results = {}
    progressTracker = ProgressTracker(len(teams))
    for team in teams:
        team_data = prescout_team(team)
        try:
            results[team] = {
                'score_avg': agr([x[1]['numeric_score'] for x in sorted(team_data.items())]),
                'year_score': {key: val['numeric_score'] for key, val in team_data.items()}
            }
        except:
            results[team] = None
        progressTracker.done()
    return results

pp.pprint(prescout_teams(['frc973', 'frc971', 'frc254', 'frc5012', 'frc492']))

20 percent done (1 out of 5)
40 percent done (2 out of 5)
60 percent done (3 out of 5)
80 percent done (4 out of 5)
100 percent done (5 out of 5)
{'frc254': {'score_avg': 12.1875,
            'year_score': {2014: 15.375, 2015: 10.625, 2016: 9.75, 2017: 13.0}},
 'frc492': {'score_avg': 8.083333333333332,
            'year_score': {2014: 8.333333333333332,
                           2015: 9.6,
                           2016: 4.166666666666667,
                           2017: 9.6}},
 'frc5012': {'score_avg': 8.10625,
             'year_score': {2014: 5.6, 2015: 8.25, 2016: 9.0, 2017: 8.25}},
 'frc971': {'score_avg': 10.575,
            'year_score': {2014: 11.4, 2015: 12.0, 2016: 12.6, 2017: 9.0}},
 'frc973': {'score_avg': 13.0,
            'year_score': {2014: 11.333333333333332,
                           2015: 9.0,
                           2016: 10.333333333333332,
                           2017: 15.75}}}


In [19]:
@lru_cache(maxsize=128)
def event_teams(team):
    return [x['key'] for x in tba.event_teams(team)]

pp.pprint(event_teams('2018caav'))

['frc1160',
 'frc2339',
 'frc2429',
 'frc2493',
 'frc2637',
 'frc2658',
 'frc294',
 'frc330',
 'frc3309',
 'frc3390',
 'frc3453',
 'frc3476',
 'frc3647',
 'frc3669',
 'frc399',
 'frc4014',
 'frc4191',
 'frc4255',
 'frc4276',
 'frc4322',
 'frc4470',
 'frc5012',
 'frc5124',
 'frc5474',
 'frc5625',
 'frc5805',
 'frc585',
 'frc5869',
 'frc597',
 'frc599',
 'frc6060',
 'frc6072',
 'frc6973',
 'frc980',
 'frc981']


In [25]:
def prescout_event(event_code):
    return sorted(
        prescout_teams(event_teams(event_code)).items(),
        key=lambda x: 0 if x[1] is None else x[1]['score_avg'])[::-1]

caavprescout = prescout_event('2018caav')

pp.pprint(caavprescout)

11 percent done (4 out of 35)
20 percent done (7 out of 35)
31 percent done (11 out of 35)
40 percent done (14 out of 35)
51 percent done (18 out of 35)
60 percent done (21 out of 35)
71 percent done (25 out of 35)
80 percent done (28 out of 35)
91 percent done (32 out of 35)
100 percent done (35 out of 35)
[('frc330',
  {'score_avg': 11.09375,
   'year_score': {2014: 10.25,
                  2015: 10.666666666666668,
                  2016: 12.416666666666668,
                  2017: 10.75}}),
 ('frc3309',
  {'score_avg': 10.523214285714285,
   'year_score': {2014: 9.4,
                  2015: 9.285714285714285,
                  2016: 10.583333333333332,
                  2017: 11.083333333333332}}),
 ('frc399',
  {'score_avg': 10.25,
   'year_score': {2014: 9.833333333333334,
                  2015: 10.1,
                  2016: 9.7,
                  2017: 10.666666666666668}}),
 ('frc3476',
  {'score_avg': 9.497916666666665,
   'year_score': {2014: 11.5,
                  2015: 10

In [24]:
caavitems = caavprescout.items()

caavitems = sorted(caavitems, key=lambda x: 0 if x[1] is None else x[1]['score_avg'])[::-1]
pp.pprint(caavitems)

[('frc330',
  {'score_avg': 11.09375,
   'year_score': {2014: 10.25,
                  2015: 10.666666666666668,
                  2016: 12.416666666666668,
                  2017: 10.75}}),
 ('frc3309',
  {'score_avg': 10.523214285714285,
   'year_score': {2014: 9.4,
                  2015: 9.285714285714285,
                  2016: 10.583333333333332,
                  2017: 11.083333333333332}}),
 ('frc399',
  {'score_avg': 10.25,
   'year_score': {2014: 9.833333333333334,
                  2015: 10.1,
                  2016: 9.7,
                  2017: 10.666666666666668}}),
 ('frc3476',
  {'score_avg': 9.497916666666665,
   'year_score': {2014: 11.5,
                  2015: 10.583333333333332,
                  2016: 10.75,
                  2017: 8.1}}),
 ('frc5012',
  {'score_avg': 8.10625,
   'year_score': {2014: 5.6, 2015: 8.25, 2016: 9.0, 2017: 8.25}}),
 ('frc2637',
  {'score_avg': 8.09375,
   'year_score': {2014: 7.75, 2015: 7.0, 2016: 8.0, 2017: 8.5}}),
 ('frc294',
  {'sco

In [26]:
team_years = lru_cache(maxsize=512)(tba.team_years)

pp.pprint(team_years('frc492'))

[2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018]


In [144]:
def team_event_report(team, event):
    level = None
    seat = None
    seed = None
    
    try:
        finals_report = event_alliances(event)
    
        for alliance in finals_report:
            if team in alliance['picks'] and alliance['status'] != 'unknown':
                #pp.pprint(alliance)
                level = 'w' if alliance['status']['status'] == 'won' else alliance['status']['level']
                seat = alliance['picks'].index(team)
                if seat == 0:
                    seat = 'AC'
    except TypeError:
        print("No finals report for {:s}".format(event))
    
    try:
        quals_seeds = event_rankings(event)
    
        for alliance in quals_seeds['rankings']:
            if alliance['team_key'] == team:
                if alliance['dq']:
                    seed = 'DQ'
                else:
                    seed = alliance['rank']
    except TypeError:
        print("No ranking port for {:s}".format(event))
    
    return {
        'level': level,
        'seat': seat,
        'seed': seed,
    }

def team_year_report(team, year):
    events = [event
              for event
              in team_events(team, year)
              if event['event_type'] <= 3]
    
    event_reports = []
    for event in events:
        event_code = str(year) + event['event_code']
        event_reports.append({
            'event_code': event['event_code'],
            'event_type': event['event_type'],
            'event_type_string': event['event_type_string'],
            'event_date': event['start_date'],
            'team_report': team_event_report(team, event_code),
        })
        
    return event_reports

def team_history(team):
    years = sorted(team_years(team))
    
    year_reports = []
    for year in years:
        year_reports.append((year, team_year_report(team, year)))
        
    return year_reports

griffengear = team_history('frc5012')
pp.pprint(griffengear)

No finals report for 2015nvlv
No finals report for 2018caav
No ranking port for 2018caav
No finals report for 2018casd
No ranking port for 2018casd
No finals report for 2018nvlv
No ranking port for 2018nvlv
[(2014,
  [{'event_code': 'casb',
    'event_date': '2014-02-28',
    'event_type': 0,
    'event_type_string': 'Regional',
    'team_report': {'level': 'sf', 'seat': 2, 'seed': 20}},
   {'event_code': 'gal',
    'event_date': '2014-04-24',
    'event_type': 3,
    'event_type_string': 'Championship Division',
    'team_report': {'level': None, 'seat': None, 'seed': 41}},
   {'event_code': 'nvlv',
    'event_date': '2014-04-03',
    'event_type': 0,
    'event_type_string': 'Regional',
    'team_report': {'level': None, 'seat': None, 'seed': 42}}]),
 (2015,
  [{'event_code': 'carm',
    'event_date': '2015-02-26',
    'event_type': 0,
    'event_type_string': 'Regional',
    'team_report': {'level': 'qf', 'seat': 1, 'seed': 23}},
   {'event_code': 'new',
    'event_date': '2015-04-2

In [268]:
class TableBuilder:
    def __init__(self, **kwargs):
        self.row_var = kwargs['row']
        self.rows = []
        self.accept_new_rows = True
        self.col_var = kwargs['col']
        self.cols = []
        self.accept_new_cols = True
        self.upper_left = ''
        
        self.cells = {}
        
        def default_generator(**kwargs):
            return ""
        self.generator = default_generator
    
    def set_generator(self, gen):
        self.generator = gen
        
    def set_rows(self, rows):
        self.rows = rows
        self.accept_new_rows = False
    
    def set_cols(self, cols):
        self.cols = cols
        self.accept_new_cols = False
        
    def set_cell(self, value, **kwargs):
        row = kwargs[self.row_var]
        col = kwargs[self.col_var]
        
        if row not in self.rows:
            if self.accept_new_rows:
                self.rows.append(row)
            else:
                raise Exception("Row does not exist and table configured to not accept new rows")
        
        if col not in self.cols:
            if self.accept_new_cols:
                self.cols.append(col)
            else:
                raise Exception("Col does not exist and table configured to not accept new cols")
        
        if row not in self.cells:
            self.cells[row] = {}
            
        self.cells[row][col] = {
            'value': value,
            'color': kwargs.get('color', None),
        }
    
    def get_cell(self, **kwargs):
        row = kwargs[self.row_var]
        col = kwargs[self.col_var]
        if row not in self.cells:
            self.cells[row] = {}
        if col not in self.cells[row]:
            self.cells[row][col] = self.generator(row=row, col=col)
        return self.cells[row][col]
    
    def _get_cell(self, row, col):
        if row not in self.cells:
            self.cells[row] = {}
        if col not in self.cells[row]:
            self.cells[row][col] = {
                'value': self.generator(row=row, col=col),
                'color': None,
            }
        return self.cells[row][col]
    
    def get_html(self):
        table_rows = []
        
        header_cells = [self.upper_left] + self.cols
        header = ''.join('<td>{:s}</td>'.format(str(cell)) for cell in header_cells)
        table_rows.append(header)
        
        for row in self.rows:
            row_cells = [self._get_cell(row, col) for col in self.cols]
            row = ['<td>{:s}</td>'.format(row)]
            for cell in row_cells:
                if cell['color']:
                    row.append('<td style="background-color:{:s}">{:s}</td>'.format(
                                    str(cell['color']),
                                    str(cell['value'])))
                else:
                    row.append('<td>{:s}</td>'.format(
                                    str(cell['value'])))
            row = ''.join(row)
            table_rows.append(row)
            
        contents = ''.join('<tr>{:s}</tr>'.format(row) for row in table_rows)
        return "<table>{:s}</table>".format(contents)

In [189]:
from IPython.display import HTML, display

def format_subreport(history, team, attribute, color_codes=None):
    table = TableBuilder(row='event', col='year')
    
    years = [report[0] for report in history]
    table.set_cols(years)
    table.set_rows(["Regional 1", "Regional 2", "Regional 3", "Champs Div"])
    
    for report in history:
        year = report[0]
        regional_events = [event for event in report[1] if event['event_type'] < 3]
        champs_events = [event for event in report[1] if event['event_type'] == 3]
        
        for i, row in enumerate(['Regional 1', 'Regional 2', 'Regional 3']):
            if len(regional_events) > i:
                if regional_events[i]['team_report'] is None:
                    table.set_cell("?", event=row, year=year)
                elif regional_events[i]['team_report'][attribute] is None:
                    table.set_cell("N/A", 
                                   color='#e6e6e6',
                                   event=row, year=year)
                else:
                    table.set_cell(regional_events[i]['team_report'][attribute],
                                   color=color_codes[regional_events[i]['team_report'][attribute]],
                                   event=row, year=year)
        
        if champs_events:
            if champs_events[0]['team_report'] is None:
                table.set_cell("?", event='Champs Div', year=year)
            elif champs_events[0]['team_report'][attribute] is None:
                table.set_cell("N/A",
                               color='#e6e6e6',
                               event='Champs Div', year=year)
            else:
                table.set_cell(champs_events[0]['team_report'][attribute],
                               color=color_codes[champs_events[0]['team_report'][attribute]],
                               event='Champs Div', year=year)
    return table.get_html()

def rank_to_color(rank):
    rank = min(rank, 60)
    r = 60 + 1 * rank
    g = 255 - 1 * rank
    b = 135 + 2 * rank
    return "#{:02x}{:02x}{:02x}".format(r, g, b)

def format_report(history, teamno):
    source = (
        "<h1>Report for {:s}</h1>".format(str(teamno)) +
        "<h3>Elims results</h3>" +
        format_subreport(history, teamno, 'level', {
            'w': '#66ff33',
            'f': '#66ff99',
            'sf': '#66ffff',
            'qf': '#66ccff',
            'N/A': '#e6e6e6',
        }) +
        "<br />" +
        "<h3>Elims alliance seat</h3>" +
        format_subreport(history, teamno, 'seat', {
            'AC': '#66ff33',
            1: '#66ff99',
            2: '#66ffff',
            3: '#66ccff',
        }) +
        "<br />" +
        "<h3>Quals seed</h3>" +
        format_subreport(history, teamno, 'seed', {
                rank: rank_to_color(rank)
                for rank
                in range(100)
        })
    )
    display(HTML(source))

format_report(griffengear, 5012)

,2014,2015,2016,2017,2018
Regional 1,sf,qf,w,sf,N/A
Regional 2,N/A,N/A,sf,N/A,N/A
Regional 3,,,,qf,N/A
Champs Div,N/A,w,N/A,qf,
,2014,2015,2016,2017,2018
Regional 1,2,1,2,2,N/A
Regional 2,N/A,N/A,AC,N/A,N/A
Regional 3,,,,1,N/A
Champs Div,N/A,3,N/A,3,
,2014,2015,2016,2017,2018


In [187]:
format_report(team_history('frc973'), 973)

No finals report for 2002sj
No ranking port for 2002sj
No finals report for 2003ca
No ranking port for 2003ca
No finals report for 2003sj
No ranking port for 2003sj
No finals report for 2004az
No ranking port for 2004az
No finals report for 2004sj
No ranking port for 2004sj
No finals report for 2005sac
No ranking port for 2005sac
No finals report for 2006arc
No ranking port for 2006arc
No finals report for 2006sj
No ranking port for 2006sj
No finals report for 2007sac
No finals report for 2007sj
No finals report for 2008nv
No finals report for 2008sj
No finals report for 2009ca
No finals report for 2009gal
No finals report for 2009nv
No finals report for 2018casf
No ranking port for 2018casf
No finals report for 2018casj
No ranking port for 2018casj


,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Regional 1,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,f,sf,f,w,f,w,sf,w,N/A
Regional 2,,N/A,N/A,,,N/A,N/A,N/A,qf,qf,sf,sf,w,qf,w,sf,N/A
Regional 3,,,,,,,,,,,,f,,,,,
Champs Div,,,,,N/A,,,N/A,,w,qf,qf,w,qf,sf,w,
,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Regional 1,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,1,1,1,1,1,1,1,1,N/A
Regional 2,,N/A,N/A,,,N/A,N/A,N/A,1,AC,1,AC,1,AC,1,AC,N/A
Regional 3,,,,,,,,,,,,1,,,,,
Champs Div,,,,,N/A,,,N/A,,2,1,2,1,1,AC,AC,
,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018


In [190]:
format_report(team_history('frc254'), 254)
format_report(team_history('frc1678'), 1678)
foformat_report(team_history('frc254'), 254)rmat_report(team_history('frc971'), 971)
format_report(team_history('frc492'), 492)

No finals report for 1999ca
No ranking port for 1999ca
No finals report for 2000ca
No ranking port for 2000ca
No finals report for 2000tx
No ranking port for 2000tx
No finals report for 2001arc
No ranking port for 2001arc
No finals report for 2001ca1
No ranking port for 2001ca1
No finals report for 2001ca2
No ranking port for 2001ca2
No finals report for 2002oh
No ranking port for 2002oh
No finals report for 2002sj
No ranking port for 2002sj
No finals report for 2003new
No ranking port for 2003new
No finals report for 2003sac
No ranking port for 2003sac
No finals report for 2003sj
No ranking port for 2003sj
No finals report for 2004new
No ranking port for 2004new
No finals report for 2004sac
No ranking port for 2004sac
No finals report for 2004sj
No ranking port for 2004sj
No finals report for 2005gal
No ranking port for 2005gal
No finals report for 2005sac
No ranking port for 2005sac
No finals report for 2005sj
No ranking port for 2005sj
No finals report for 2006new
No ranking port fo

,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Regional 1,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,w,w,f,w,w,w,w,w,N/A
Regional 2,,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,w,w,N/A,w,w,w,w,w,N/A
Regional 3,,,,,,,,N/A,N/A,,,,,w,,w,,,,
Champs Div,,,N/A,,N/A,N/A,N/A,N/A,N/A,N/A,N/A,w,w,f,sf,w,qf,f,w,
,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Regional 1,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,1,AC,1,1,AC,1,1,AC,N/A
Regional 2,,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,1,AC,N/A,AC,1,AC,1,AC,N/A
Regional 3,,,,,,,,N/A,N/A,,,,,1,,AC,,,,
Champs Div,,,N/A,,N/A,N/A,N/A,N/A,N/A,N/A,N/A,AC,AC,1,1,AC,AC,1,1,
,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018


No finals report for 2005sac
No ranking port for 2005sac
No finals report for 2006sac
No ranking port for 2006sac
No finals report for 2007sac
No finals report for 2008sac
No finals report for 2009sac
No finals report for 2011sac
No finals report for 2018cada
No ranking port for 2018cada
No finals report for 2018cafr
No ranking port for 2018cafr
No finals report for 2018utwv
No ranking port for 2018utwv


,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Regional 1,N/A,N/A,N/A,N/A,N/A,qf,N/A,qf,w,w,w,w,w,N/A
Regional 2,,,,,,,,w,qf,w,w,w,w,N/A
Regional 3,,,,,,,,,,f,w,w,w,N/A
Champs Div,,,,,,,qf,qf,w,w,w,w,w,
,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Regional 1,N/A,N/A,N/A,N/A,N/A,AC,N/A,1,AC,1,AC,1,AC,N/A
Regional 2,,,,,,,,AC,1,AC,1,AC,AC,N/A
Regional 3,,,,,,,,,,AC,1,AC,AC,N/A
Champs Div,,,,,,,1,1,AC,AC,1,1,1,
,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018


No finals report for 2002sj
No ranking port for 2002sj
No finals report for 2003cur
No ranking port for 2003cur
No finals report for 2003sac
No ranking port for 2003sac
No finals report for 2003sj
No ranking port for 2003sj
No finals report for 2004gal
No ranking port for 2004gal
No finals report for 2004sj
No ranking port for 2004sj
No finals report for 2005sj
No ranking port for 2005sj
No finals report for 2006gal
No ranking port for 2006gal
No finals report for 2006sj
No ranking port for 2006sj
No finals report for 2007sj
No finals report for 2008sj
No finals report for 2009gal
No finals report for 2009sj
No finals report for 2010new
No finals report for 2018casf
No ranking port for 2018casf
No finals report for 2018casj
No ranking port for 2018casj


,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Regional 1,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,w,sf,w,sf,w,sf,w,w,N/A
Regional 2,,N/A,,,,,,,,,w,qf,w,f,f,qf,N/A
Regional 3,,,,,,,,,,,,,,,,,
Champs Div,,N/A,N/A,,N/A,,,N/A,N/A,,qf,,f,sf,f,f,
,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Regional 1,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,AC,AC,1,AC,AC,1,AC,1,N/A
Regional 2,,N/A,,,,,,,,,AC,AC,AC,1,AC,AC,N/A
Regional 3,,,,,,,,,,,,,,,,,
Champs Div,,N/A,N/A,,N/A,,,N/A,N/A,,1,,AC,AC,AC,1,
,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018


No finals report for 2001ca2
No ranking port for 2001ca2
No finals report for 2002ca
No ranking port for 2002ca
No finals report for 2002wa
No ranking port for 2002wa
No finals report for 2003sj
No ranking port for 2003sj
No finals report for 2003wa
No ranking port for 2003wa
No finals report for 2004gal
No ranking port for 2004gal
No finals report for 2004or
No ranking port for 2004or
No finals report for 2005gal
No ranking port for 2005gal
No finals report for 2005or
No ranking port for 2005or
No finals report for 2006new
No ranking port for 2006new
No finals report for 2006or
No ranking port for 2006or
No finals report for 2006wa
No ranking port for 2006wa
No finals report for 2007or
No finals report for 2008or
No finals report for 2008wa
No finals report for 2009arc
No finals report for 2009wa
No finals report for 2013wase
No finals report for 2018waamv
No ranking port for 2018waamv
No finals report for 2018wasno
No ranking port for 2018wasno


,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Regional 1,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,sf,N/A,w,N/A,qf,qf,qf,sf,N/A
Regional 2,,N/A,N/A,,,N/A,,N/A,,,,,,qf,sf,N/A,sf,N/A
Regional 3,,,,,,,,,,,,,,f,w,,w,
Champs Div,,,,N/A,N/A,N/A,,,N/A,,,qf,N/A,,w,,f,
,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Regional 1,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,2,N/A,2,N/A,AC,1,1,1,N/A
Regional 2,,N/A,N/A,,,N/A,,N/A,,,,,,AC,AC,N/A,1,N/A
Regional 3,,,,,,,,,,,,,,1,1,,AC,
Champs Div,,,,N/A,N/A,N/A,,,N/A,,,AC,N/A,,2,,AC,
,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018


In [191]:
format_report(team_history('frc399'), 399)

No finals report for 2000ca
No ranking port for 2000ca
No finals report for 2001ca1
No ranking port for 2001ca1
No finals report for 2002ca
No ranking port for 2002ca
No finals report for 2002new
No ranking port for 2002new
No finals report for 2003ca
No ranking port for 2003ca
No finals report for 2003gal
No ranking port for 2003gal
No finals report for 2004arc
No ranking port for 2004arc
No finals report for 2004ca
No ranking port for 2004ca
No finals report for 2005gal
No ranking port for 2005gal
No finals report for 2005nv
No ranking port for 2005nv
No finals report for 2006gal
No ranking port for 2006gal
No finals report for 2006md
No ranking port for 2006md
No finals report for 2006nv
No ranking port for 2006nv
No finals report for 2007ca
No finals report for 2007new
No finals report for 2007pa
No finals report for 2008ca
No finals report for 2008gal
No finals report for 2008sdc
No finals report for 2009ca
No finals report for 2009co
No finals report for 2009cur
No finals report 

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Regional 1,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,sf,w,f,w,qf,sf,sf,N/A
Regional 2,,,,,,,N/A,N/A,N/A,N/A,N/A,f,f,N/A,sf,w,qf,f,N/A
Regional 3,,,,,,,,,,,,N/A,qf,sf,sf,sf,qf,f,N/A
Champs Div,,,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,sf,N/A,N/A,qf,qf,,qf,
,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Regional 1,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,1,1,AC,1,1,AC,AC,N/A
Regional 2,,,,,,,N/A,N/A,N/A,N/A,N/A,1,1,N/A,1,AC,1,1,N/A
Regional 3,,,,,,,,,,,,N/A,AC,AC,1,AC,1,AC,N/A
Champs Div,,,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,AC,N/A,N/A,AC,2,,1,
,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018


In [283]:
def opr_percentile_at_event(team, event):
    try:
        percentiles = event_oprs(event)['oprs']
        percentiles = [(opr, team) for team, opr in percentiles.items()]
        percentiles = sorted(percentiles)
        percentiles = [team for opr, team in percentiles]
        position = percentiles.index(team)
        return float(position + 1) / len(percentiles)
    except TypeError:
        return None
    except ValueError:
        return None

def opr_percentile_at_all_events(team, year):
    events = [
        str(year) + event['event_code']
        for event
        in team_events(team, year)
        if event['event_type'] <= 3
    ]
    
    return [
        (event, opr_percentile_at_event(team, event))
        for event
        in events
    ]

def avg(lst):
    return sum(lst) / len(lst)

def stddev(lst):
    mean = avg(lst)
    diffs = [(mean - x) ** 2 for x in lst]
    return avg(diffs) ** 0.5

def team_opr_strength(team):
    percentiles = []
    for year in [2017, 2016, 2015]:
        for event, percentile in opr_percentile_at_all_events(team, year):
            if percentile:
                percentiles.append(percentile)
    if percentiles:
        return {
            'average': avg(percentiles),
            'stddev': stddev(percentiles),
        }
    else:
        return {
            'average': None,
            'stddev': None,
        }

"""
for team in ['frc973', 'frc294', 'frc492', 'frc3309', 'frc360', 'frc5499', 'frc1700', 'frc1458']:
    print(team)
    percentiles = []
    for year in [2017, 2016, 2015]:
        for event, percentile in opr_percentile_at_all_events(team, year):
            percentiles.append(percentile)
            print("{:s}: {:2.0f}".format(event, percentile * 100))
        print()
    print("Avg: {:2.2f} stddev: {:2.2f}".format(avg(percentiles), stddev(percentiles)))
    print('\n\n\n')
"""
None

In [286]:
def prescout_event(event):
    teams = event_teams(event)
    opr_strengths = [(team, team_opr_strength(team)) for team in teams]
    return opr_strengths

def stats_to_color(stats):
    if stats['average'] is None:
        return '#ffffff'
    color = int(stats['average'] * 100)
    lightness = 50 + int(stats['stddev'] * 150)
    return "hsl({:d},{:d}%,{:d}%)".format(color, 100, lightness)
    

def format_event_report(report):
    report = sorted(report, key=lambda x: x[1]['average'] if x[1]['average'] else 0)[::-1]
    table = TableBuilder(row='team', col='stat')
    
    table.set_cols(["OPR P", "OPR P SD"])
    
    for team, stats in report:
        table.set_cell(stats['average'], color=stats_to_color(stats), team=team, stat='OPR P')
        table.set_cell(stats['stddev'], color=stats_to_color(stats), team=team, stat='OPR P SD')
    
    return table.get_html()

for event in ['2018caav', '2017calb', '2017cc']:
    display(HTML(
            "<h1>Event: {:s}</h1>".format(event) +
            "Color determined by OPR Percentile.  Saturation determined by OPR Percentile Standard Deviation" +
            format_event_report(prescout_event(event))
    ))

,OPR P,OPR P SD
frc330,0.9668349563793398,0.03124240715745202
frc3309,0.8896506277949777,0.17524072628393625
frc294,0.8670341507263599,0.08578868905965303
frc399,0.7923372236674656,0.18007240955607115
frc3476,0.7679126641352716,0.1945517142696203
frc2637,0.7626578129672084,0.19091289503213651
frc5124,0.700786698504789,0.2534131125461936
frc3647,0.6973005744305883,0.2705987113412613
frc1160,0.6746760184260184,0.13288523957876713
frc5805,0.6739277389277389,0.2608072969136756


,OPR P,OPR P SD
frc330,0.9668349563793398,0.03124240715745202
frc3309,0.8896506277949777,0.17524072628393625
frc294,0.8670341507263599,0.08578868905965303
frc2637,0.7626578129672084,0.19091289503213651
frc2486,0.7425903828329473,0.2261404054148349
frc3970,0.7380011226078391,0.11741855503386818
frc207,0.7348538677839063,0.19085093733964464
frc3512,0.7281156568158657,0.26803389057037996
frc696,0.7256975676394263,0.19381212099709935
frc4201,0.7027959609704771,0.22824708551895712


,OPR P,OPR P SD
frc254,0.9920594362406211,0.011328505154128693
frc1678,0.9811761104514763,0.013529528173670268
frc973,0.9624740934187422,0.022835540855692893
frc971,0.9598866647800534,0.040414210035810784
frc1538,0.9340506307073976,0.06776754903068292
frc3309,0.8896506277949777,0.17524072628393625
frc701,0.834476912381749,0.1344336751129024
frc399,0.7923372236674656,0.18007240955607115
frc1671,0.784792149241849,0.20567955048851894
frc846,0.7697298133153397,0.0649797992662614
